In [22]:
DATABASE='googol'
HOST='127.0.0.1'
USERNAME='kenjiwang'
PASSWORD='5517007'
PORT='5432'

In [23]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from numpy.linalg import norm
import numpy as np
import psycopg2

In [24]:
conn = psycopg2.connect(database=DATABASE,
	host=HOST,
	user=USERNAME,
	password=PASSWORD,
	port=PORT
)

cursor = conn.cursor()

In [25]:
model = AutoModel.from_pretrained('nvidia/NV-Embed-v2', trust_remote_code=True)

In [18]:
def search_googol(query):
	query_formatted = query.replace('\'', '\'\'')
	query_emb = model.encode([query], instruction="", max_length=4096, normalize_embeddings=True).tolist()
	pg_query = f"SELECT * FROM websites ORDER BY description_emb <=> '{query_emb}', title_emb <=> '{query_emb}' LIMIT 10;"

	cursor.execute(pg_query)
	many_rows = cursor.fetchmany(10)

	for row in many_rows:
		print(row)

In [19]:
search_googol("British news")

(55, 'bbc.co.uk', 'BBC - Home', 'The best of the BBC, with the latest news and sport headlines, weather, TV & radio highlights and much more from across the whole of BBC Online.', '[-0.10518994,-0.033777248,-0.69257593,-0.119483255,-0.08689426,-0.094562024,-0.618515,0.4088498,0.21463479,0.2325919,-0.07443373,-0.30983737,0.41127437,-0.26919878,-0.70556587,-0.06764819,-0.7404579,-1.2402654,-0.53849864,-0.2000086,0.60056484,-0.33325103,-0.5526745,-0.54456395,0.1481525,1.0504662,0.3639962,0.5860761,1.2061927,0.6698942,0.49652246,-0.72129977,0.77277863,-1.5463458,0.6408407,-0.72361994,0.17834374,-0.32042894,0.042046595,-0.28511015,0.36606228,-0.7351871,0.7065457,-1.1870754,-0.99351335,-0.29804736,-0.42940354,-0.026608318,-0.48090148,-0.3036857,-0.12416148,0.63763386,0.6240512,-0.88751817,-0.07892336,0.29605982,-0.5387201,0.51995647,-0.31561273,0.24478687,0.40502018,0.07048477,0.6539167,-1.4064237,-0.2359122,0.41823944,0.44148436,-0.94888186,0.24236917,-0.6068877,-0.73138297,0.09347472,0.312